# Operator overloading

|       Category       |          Trait             |             Operator              |
| -------------------- | -------------------------- | --------------------------------- |
| Unary operators      | `std::ops::Neg`            | `-x   `                           |
|                      | `std::ops::Not`            | `!x   `                           |
| Arithmetic operators | `std::ops::Add`            | `x + y`                           |
|                      | `std::ops::Sub`            | `x - y`                           |
|                      | `std::ops::Mul`            | `x * y`                           |
|                      | `std::ops::Div`            | `x / y`                           |
|                      | `std::ops::Rem`            | `x % y`                           |
| Bitwise operators    | `std::ops::BitAnd`         | `x & y`                           |
|                      | `std::ops::BitOr `         | `x \| y`                          |
|                      | `std::ops::BitXor`         | `x ^ y`                           |
|                      | `std::ops::Shl   `         | `x << y`                          |
|                      | `std::ops::Shr   `         | `x >> y`                          |
| Compound assignment arithmetic operators | `std::ops::AddAssign`      | `x += y`      |
|                      | `std::ops::MulAssign`      | `x *= y`                          |
|                      | `std::ops::SubAssign`      | `x -= y`                          |
|                      | `std::ops::DivAssign`      | `x /= y`                          |
|                      | `std::ops::RemAssign`      | `x %= y`                          |
| Compound assignment bitwise operators    | `std::ops::BitAndAssign`   | `x &= y`      |
|                      | `std::ops::BitOrAssign `   | `x \| = y`                        |
|                      | `std::ops::BitXorAssign`   | `x ^= y`                          |
|                      | `std::ops::ShlAssign   `   | `x <<= y`                         |
|                      | `std::ops::ShrAssign   `   | `x >>= y`                         |
| Comparison           | `std::cmp::PartialEq   `   | `x == y`, `x != y`                |
|                      | `std::cmp::PartialOrd`     | `x < y,  x <= y,  x > y,  x >= y` |
| Indexing             | `std::ops::Index`          | `x[y],  &x[y]`                    |
|                      | `std::ops::IndexMut`       | `x[y] = z,  &mut x[y]`            |

In [2]:
#[derive(Debug, Clone, Copy)]
struct Complex<T> {
    re: T,
    im: T,
}

In [3]:
let z: Complex<f64> = Complex { re: 1.0, im: 2.0 };

In [4]:
z + z

Error: an implementation of `Add` might be missing for `Complex<f64>`

## Implementing `Add` trait

In [5]:
mod explain {
    trait Add<RHS=Self> {
        type Output;
        fn add(self, rhs: RHS) -> Self::Output;
    }
}

In [6]:
use std::ops::Add;

impl<T> Add for Complex<T> 
where T: Add<Output = T> {
    type Output = Self;   
     
    fn add(self, rhs: Complex<T>) -> Self::Output {
        Complex {
            re: self.re + rhs.re,
            im: self.im + rhs.im,
        }
    }
}

In [7]:
z + z

Complex { re: 2.0, im: 4.0 }

## Implementing `Neg` trait

In [8]:
mod explain {
    trait Neg {
        type Output;
        fn neg(self) -> Self::Output;
    }
}

In [9]:
-z

Error: an implementation of `std::ops::Neg` might be missing for `Complex<f64>`

In [11]:
use std::ops::Neg;

impl<T> Neg for Complex<T>
where T: Neg<Output = T> {
    type Output = Self;
    
    fn neg(self) -> Self::Output {
        Complex {
            re: -self.re,
            im: -self.im,
        }
    }
}

In [12]:
-z

Complex { re: -1.0, im: -2.0 }

## Compound Assignment Operators

In [13]:
mod explain {
    trait AddAssign<RHS=Self> {
        fn add_assign(&mut self, rhs: RHS);
    }
}

In [14]:
use std::ops::AddAssign;

impl<T> AddAssign for Complex<T>
where T: AddAssign<T> {
    fn add_assign(&mut self, rhs: Complex<T>) {
        self.re += rhs.re;
        self.im += rhs.im;
    }
}

In [17]:
let mut c = Complex { re: 1.0, im: 2.0 };

c += Complex { re: 3.0, im: 4.0 };

c

Complex { re: 4.0, im: 6.0 }

## Equivalence Comparisons

### PartialEq

In [18]:
mod explain {
    trait PartialEq<RHS=Self> 
    where RHS: ?Sized
    {
        fn eq(&self, other: &RHS) -> bool;
        
        fn ne(&self, other: &RHS) -> bool { !self.eq(other) }
    }
}

In [20]:
use std::cmp::PartialEq;

impl<T: PartialEq> PartialEq for Complex<T> {
    fn eq(&self, other: &Complex<T>) -> bool {
        self.re == other.re && self.im == other.im
    }
}

In [21]:
z == z

true

In [22]:
z != c

true

### PartialOrd

In [2]:
mod explain {
    enum Ordering {
        Less,
        Equal,
        Greater,
    }

    trait PartialOrd<RHS=Self>: PartialEq<RHS> 
    where 
        RHS: ?Sized 
    {
        fn partial_cmp(&self, other: &RHS) -> Option<Ordering>;
        
        fn lt(&self, other: &RHS) -> bool { 
            match self.partial_cmp(other) {
                Some(Ordering::Less) => true,
                _ => false,
            }
        }
        
        fn le(&self, other: &RHS) -> bool { 
            match self.partial_cmp(other) {
                Some(Ordering::Less) | Some(Ordering::Equal) => true,
                _ => false,
            }
        }
        
        fn gt(&self, other: &RHS) -> bool { 
            match self.partial_cmp(other) {
                Some(Ordering::Greater) => true,
                _ => false,
            }
        }
        
        fn ge(&self, other: &RHS) -> bool { 
            match self.partial_cmp(other) {
                Some(Ordering::Greater) | Some(Ordering::Equal) => true,
                _ => false,
            }
        }
    }

    // trait for types that can be totally ordered
    trait Ord: Eq + PartialOrd<Self> {
        fn cmp(&self, other: &Self) -> Ordering;
    }
}

In [4]:
#[derive(PartialEq, Debug)]
struct Interval<T> {
    lower: T, // inclusive
    upper: T  // exclusive 
}

In [7]:
use std::cmp::{Ordering, PartialOrd};

impl<T: PartialOrd> PartialOrd for Interval<T> {
    fn partial_cmp(&self, other: &Interval<T>) -> Option<Ordering> {
        if self == other {
            Some(Ordering::Equal)
        } else if self.lower >= other.upper {
            Some(Ordering::Greater)
        } else if self.upper <= other.lower {
            Some(Ordering::Less)
        } else {
            None
        }            
    }
}

In [9]:
assert!(Interval { lower: 10, upper: 20 } <  Interval { lower: 20, upper: 40 });
assert!(Interval { lower: 7,  upper: 8  } >= Interval { lower: 0,  upper: 1  });
assert!(Interval { lower: 7,  upper: 8  } <= Interval { lower: 7,  upper: 8  });

// Overlapping intervals aren't ordered with respect to each other.
let left  = Interval { lower: 10, upper: 30 };
let right = Interval { lower: 20, upper: 40 };
assert!(!(left < right));
assert!(!(left >= right));

### Indexing

In [2]:
mod explain {
    
    trait Index<Idx> {
        type Output: ?Sized;
        fn index(&self, index: Idx) -> &Self::Output;
    }

    trait IndexMut<Idx>: Index<Idx> {
        fn index_mut(&mut self, index: Idx) -> &mut Self::Output;
    }
}

In [6]:
struct Image<P> {
    width: usize,
    pixels: Vec<P>,
}

impl<P: Default + Copy> Image<P> {

    /// Create a new image with the given dimensions.
    fn new(width: usize, height: usize) -> Self {
        Image {
            width: width,
            pixels: vec![P::default(); width * height],
        }
    }
}

impl<P> std::ops::Index<usize> for Image<P> {
    type Output = [P];
    
    fn index(&self, row: usize) -> &[P] {
        let start = row * self.width;
        &self.pixels[start..start + self.width]
    }
}

impl<P> std::ops::IndexMut<usize> for Image<P> {
    fn index_mut(&mut self, row: usize) -> &mut [P] {
        let start = row * self.width;
        &mut self.pixels[start..start + self.width]
    }
}

In [8]:
let mut bitmap = Image::new(2, 2);

bitmap[0][0] = 1;
bitmap[0][1] = 2;

assert!(bitmap[0][0] == 1);
assert!(bitmap[0][1] == 2);